# 本部分主要利用之前保存好的model进行对原视频数据的预测情况进行测试

In [ ]:
from keras.preprocessing.image import img_to_array
from keras.models import load_model
import numpy as np
import imutils
import cv2
import os

In [ ]:
model = load_model("/Users/laq66/Desktop/data_model/act_CNN_1.hdf5")
camera = cv2.VideoCapture("/Users/laq66/Desktop/data_model/1_6_0_1.mp4")
imagefile = r"/Users/laq66/Desktop/data_model/resultv2"
count=1
fourcc = cv2.VideoWriter_fourcc('X','V','I','D')
out = cv2.VideoWriter('/Users/laq66/Desktop/data_model/output.avi',fourcc, 2.0, (640,480))


设置的显示预测结果的阈值是0.97，小于这个值不输出预测结果。保存最后的视频，本部分是慢放的。

In [ ]:
while camera.isOpened():
    (grabbed, frame) = camera.read()

    
# grabbed和frame是read的两个返回值，grabbed是布尔类型的返回值，如果读取帧是正确的返回True，当文件读到结尾的时候返回False
# frame是每一帧的图像，是一个三维矩阵   

    frame = imutils.resize(frame, height=480,width=640) #把图像宽度重新指定为300像素
    #gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) #因为模型训练是对灰度图像处理，所以这里要转换成灰度图像
    frame2 = frame.copy() 
    frameClone = frame.copy()    #重新克隆frame，用于接下来绘制边界框
    frame2 = cv2.resize(frame2, (64, 48))   # 将候选区域变成28*28像素，接下来的操作都是便于roi用于CNN学习
    frame2 = frame2.astype("float") / 255.0
    frame2 = img_to_array(frame2)
    frame2 = np.expand_dims(frame2, axis = 0)   # 扩展数组，在最开始的位置新加一个维度，便于Keras识别
        
    (a,b,c,d,e,f,g,h) = model.predict(frame2)[0]    # 看概率，哪个概率大，就说明是笑还是不笑
    labels_f={'right_hand_hook':a,'right_hand_cross':b,'left_hand_uppercut':c,'right_hand_uppercut':d
    ,'left_kick_attack':e,'right_kick_attack':f,'turn_left':g,'turn_right':h}
        
    label = max(labels_f,key=labels_f.get)
    pre = max(model.predict(frame2)[0])/sum(model.predict(frame2)[0])
    print(pre)
    if(pre>0.97):
        cv2.putText(frameClone, "result: "+label+' pro:'+str(round(pre,3)), (20, 20), cv2.FONT_HERSHEY_SIMPLEX, 
              1, (0, 0, 255), 2)        
    
    else:
        cv2.putText(frameClone, "result: None", (20, 20), cv2.FONT_HERSHEY_SIMPLEX, 
              1, (0, 0, 255), 2)        
        
    
    count+=1
#         cv2.rectangle(frameClone, (146, 124), (146 + 300,  + fH),
#                      (0, 0, 255), 2)   
    out.write(frameClone)    
    cv2.imshow('result',frameClone)  # 展示包含label的微笑判断
    #cv2.imwrite(imagefile+'/'+str(count)+'.jpg',frameClone)
    
    if cv2.waitKey(1) & 0xFF == ord("q"):  # 如果 ‘q’被按下了，停止循环
        break
    elif grabbed == False:
        break
        
camera.release()  # 清空相机
out.release()
cv2.destroyAllWindows()  # 关闭所有window